# LLM Fine-tuning



### Installation

In [ ]:
!pip install transformers datasets bitsandbytes peft trl accelerate wandb --upgrade -qqq

In [ ]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 1

### Mount + libraries

- 세션 초기화 할때 마다 여기 셀들을 실행해주세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/hackertone/'

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
import numpy as np
import json
import re
from vllm import LLM, SamplingParams
import triton

## Dataset 전처리

In [ ]:
def prompts(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(
f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>{example['instruction'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{example['output'][i]}<|eot_id|>"""
        )
    return prompt_list

In [ ]:
with open(path + "craw.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 태그 파싱
pattern = r'<STYLE (style="([^"]+)")?(primary="([^"]+)")?(tone="([^"]+)")?>([^<]+)</STYLE>'

processed_data = []
result = {'instruction': [], 'output': []}
for data in data:
    match = re.match(pattern, data)
    if match:
        if match.group(2):
            result["instruction"].append(f"{match.group(2)} 스타일의 문장을 만들어줘.")
        if match.group(4):
            result["instruction"].append(f"{match.group(4)} 어체의 문장을 만들어줘.")
        if match.group(6):
            result["instruction"].append(f"{match.group(6)} 어체의 문장을 만들어줘.")
        result["output"].append(f"{match.group(7)}")

processed_data = result
df = pd.DataFrame(processed_data)
from datasets import Dataset
dataset = Dataset.from_pandas(df)

## Modeling


### 모델 불러오기

In [ ]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id, **{"low_cpu_mem_usage": True})
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/55.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

###LoRA

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha = 16,
    lora_dropout = 0.1,
    target_modules=["q_proj", "k_proj", "v_proj"],
    # "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"
    bias="none",
    task_type="CAUSAL_LM",
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


### Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

In [ ]:
train_data = dataset

### Fine-Tuning

In [ ]:
lora_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=256,
    tokenizer=tokenizer,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        # max_steps=200,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        optim="paged_adamw_8bit",
        warmup_steps=1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=50,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=prompts,
)

lora_trainer.train()

In [ ]:
lora_model = "lora_model"
lora_trainer.model.save_pretrained(lora_model)
lora_trainer.tokenizer.save_pretrained(lora_model)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto')
model = PeftModel.from_pretrained(model, lora_model, device_map='auto')

model = model.merge_and_unload()
model.save_pretrained('llama3-3b-bllossom-ERAI')
tokenizer.save_pretrained('llama3-3b-bllossom-ERAI')

## 추론
- 실행 전에 세션 초기화 해주세요.
- 이후 위에 Mount+Libraries 셀을 실행해주세요.

In [ ]:
base_model = 'llama3-3b-bllossom-ERAI'
llm = LLM(model=base_model,
          max_model_len=55000,
          gpu_memory_utilization=0.9,
          dtype="float16")

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

INFO 11-30 15:31:07 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 11-30 15:31:17 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
WARNING 11-30 15:31:17 arg_utils.py:1013] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-30 15:31:17 config.py:1136] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-30 15:31:17 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llama3-8b-bllossom-ERAI-ver-test', speculative_config=None, tokenizer='llama3-8b-bllossom-ERAI-ver-test', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=55000, download_dir=None, 

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 11-30 15:31:22 model_runner.py:1077] Loading model weights took 6.0160 GB
INFO 11-30 15:31:23 worker.py:232] Memory profiling results: total_gpu_memory=22.17GiB initial_memory_usage=6.26GiB peak_torch_memory=7.19GiB memory_usage_post_profile=6.28GiB non_torch_memory=0.26GiB kv_cache_size=12.50GiB gpu_memory_utilization=0.90
INFO 11-30 15:31:23 gpu_executor.py:113] # GPU blocks: 7316, # CPU blocks: 2340
INFO 11-30 15:31:23 gpu_executor.py:117] Maximum concurrency for 55000 tokens per request: 2.13x
INFO 11-30 15:31:27 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-30 15:31:27 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

In [ ]:
instruction = "프론트엔드에 대해 리뷰 글 작성해줘."

messages = [
    {
      "role": "system",
      "content": '''당신은 훌륭한 글 작성 도우미 챗봇입니다.
       사용자의 스타일(userStyle10)로 작성하며 요청을 들어주세요.'''
    },
    {
      "role": "user",
      "content": instruction
    },
]

prompt_message = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
)

eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

outputs = llm.generate(prompt_message,
                       SamplingParams(stop_token_ids=eos_token_id, temperature=0.8, top_p=0.95,max_tokens=1024))

for output in outputs:
    propt = output.prompt
    generated_text = output.outputs[0].text
    print(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:29<00:00, 29.22s/it, est. speed input: 3.08 toks/s, output: 35.05 toks/s]

순수한 프론트엔드 개발자라면 크게 관심을 가지지 않았던 개발 스택이지만, 이번 프로젝트를 통해 다양한 기술 스택을 점진적으로 체득하게 되었던 프로젝트입니다.  구체적으로은 아래와 같은 프로젝트로, 다양한 기술적 challanges를 경험하게 되었고, 앞으로도 더 많은 경험을 기대하며 한 단계의 기술 수준을 높이기 위해 노력하도록 하겠습니다.  이번 프로젝트에서 포기한 기술들을 정리해본 부분도 포함하고 있습니다. 
리뷰 글 작성 후에도 프로젝트에 대한 더 많은 이야기가 있으면서, 이 글을 통해 더 많은 많은 사람들을 정보를 공유해주고 싶다는 마음으로 이 글을 작성하려고 합니다. 
이 프로젝트에서 사용한 기술 스택을 넣어주신 분들에게 감사드립니다. 
이 글에서 팀원들이 언급한 technologies 에 대한 대략적인 설명을 해보겠습니다. 
1. React와 Next.js : React를 사용하여 더 빠르고 안정적인 Web App을 개발하고자 할 때, Next.js를 사용했습니다.  사용법
React의 standard library 에서 Already Built-in에서 Next.js를 사용할 수 있습니다.  Next.js는 React에서 구현된 기존의 server-side rendering (SSR)과 static site generation (SSG) 기술을 활용해 단순한 HTML을 생성하고, 더 나아가 Next.js server를 통해 데이터를 서버에서 실행시켜 온라인으로 접근할 수 있도록 rendering이 이루어집니다.  Next.js는 또한 정적 스탬프 (Static Site) 기능을 제공하여, 빌드 후 CDN에 배포하고 다양한 호스트와 구현이 가능합니다.  Next.js는도 사실 React 기반으로 공부하고 있습니다. React를 먼저 배워보시면 Next.js를 쉽게 이해할 수 있게 됩니다. 
2. TypeScript : React 애플리케이션을 작성하면서 TypeScript를 함께 사용했습니다. 
TypeScript를 사용하면, 변수, 함수, 클래스 